In [1]:
from keras.preprocessing.image import ImageDataGenerator


from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import cv2
import os
import pandas as pd
import numpy as np
import shutil

from sklearn.metrics import accuracy_score
from google.colab import drive
from tqdm.notebook import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#drive.mount('/gdrive')
try:
    os.mkdir("transform_img")
except:
    print('directroy is already exist')
finally:
  os.mkdir("useable_data")
# 이거 수정하십쇼
base_path = "/content/drive/My Drive/text_to_image"

In [4]:
df = pd.read_excel(os.path.join(base_path, 'text_data.xlsx'), index=None)
df = df.dropna()
df

,number,filenames,text
0,0,0000.jpg,흐린 날에 해변에 파도가 침. 하늘에 한곳에선 구멍이 뚫린 것처럼 구름이 개여있음....
1,1,0001.jpg,어둡고 흐린 하늘 아래 산과 평야의 모습이 보임. 어두운 흙 위에는 녹색의 풀이 보임
2,2,0002.jpg,"모래사장이 펼쳐진 해변에서 넓고 광활한 바다를 보고 있음. 하늘은 옥색, 바다는 청..."
3,3,0003.jpg,"나무와 풀들이 자라난 바위섬에서 하늘과 바다, 땅을 동시에 보여줌. 하늘은 맑아 짙..."
4,4,0004.jpg,"섬 연안의 바다위, 배에서 섬을 촬영함. 하늘은 맑고 푸르며, 섬은 녹색의 나무로 ..."
...,...,...,...
2908,2908,2908.jpg,만년설 산과 큰 호수
2909,2909,2909.jpg,구름이 가득한 하늘과 산 절벽이 강에 거울처럼 비춰지고 있다
2910,2910,2910.jpg,햇빛이 비춰지고 있는 산에 폭포와 강
2911,2911,2911.jpg,푸른 하늘아래 바위가 듬성듬성 있는 황야


In [7]:
for filename in tqdm(df['filenames']):
    shutil.copy(os.path.join(base_path, f'filtered_data/{filename}'), os.path.join('useable_data', filename))

In [28]:
path = r"/content/drive/My Drive/text_to_image/transform_img"
len(os.listdir(path)) == len(df) * 20

True

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

already_transformed = os.listdir('/content/drive/My Drive/text_to_image/transform_img')

for n in tqdm(df.index): 
    #df['filenames']

    filename = df['filenames'][n]

    if filename in already_transformed:
        print(filename + ' is already exist')
        continue

    route = f'/content/drive/My Drive/text_to_image/filtered_data/{filename}'

    img = load_img('{}'.format(route))  # PIL 이미지

    x = img_to_array(img)  
    x = x.reshape((1,) + x.shape)  
    i = 0

    # 원본 img 파일 포함 저장
    ex_img = cv2.imread(('{}'.format(route)), cv2.IMREAD_COLOR)
    cv2.imwrite(('/content/drive/My Drive/text_to_image/transform_img/{}').format(filename), ex_img)

    for batch in datagen.flow(x, batch_size=32, save_to_dir='/content/drive/My Drive/text_to_image/transform_img',save_prefix=filename[:-4],save_format='jpg'):
        i += 1
        if i > 18:
            break  # 이미지 20장 생성

In [25]:
listdir = sorted(os.listdir('/content/drive/My Drive/text_to_image/transform_img'))

d = {fname: sum([int(f.startswith(fname[:-4])) for f in listdir])  for fname in df['filenames']}

In [26]:
for k, v in d.items():
    if v != 20:
        print(k, v)

In [39]:
# os.remove(f'/content/drive/My Drive/text_to_image/transform_img/2012_0_9711.jpg')
# shutil.copy('/content/drive/My Drive/text_to_image/transform_img/2599.jpg', '/content/drive/My Drive/text_to_image/transform_img/2599_0_2132.jpg')

'/content/drive/My Drive/text_to_image/transform_img/2599_0_2132.jpg'

In [23]:
filenames = sorted(df['filenames'].apply(lambda x: x[:-4]))

for f in listdir:
    if f[:4] not in filenames:
        os.remove(f'/content/drive/My Drive/text_to_image/transform_img/{f}')

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
from IPython.display import Image
Image('/content/transform_img/_0_8331.jpg')

In [ ]:
import numpy as np

from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image

class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size, tokenizer, max_len, image_shape, shuffle=True, x_is_image=False):
        self.X = X
        self.y = y
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_len = max_len
        self.image_shape = image_shape
        self.shuffle = shuffle
        self.x_is_image = x_is_image
        self.on_epoch_end()
            
    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))
        
    def __getitem__(self, index):
        index = index * self.batch_size

        x = self.X[index: index + self.batch_size]
        y = self.y[index: index + self.batch_size]

        if x_is_image:
            X = []
            for i in x:
                image = Image.open(i)
                image.resize((64, 64))
                X.append(np.array(image))
            
            x = np.array(x)
        
        return x, y